In [1]:
from lib.solver_tools.solver_client import SolverClient
from lib.solver_tools import Task
from lib.solver_tools.utils import make_task_comparison_link
import json
from copy import deepcopy
from lib.common_tools.read_token import read_solver_apikey
import os

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import python_lib.viz as my_viz
import python_lib.utils as my_utils

In [3]:
my_apikey = read_solver_apikey()
solver_client = SolverClient(my_apikey)

In [4]:
request_path = '../test_data/inputs/medium_test_1/request.json'

In [6]:
task_id = '34000a30-948d94b2-8665fb47-a7eb3b85'
initial_request = solver_client.get_request(task_id)

In [20]:
def transform_request(initial_request):
    assert len(initial_request['depots']) == 1
    
    locations = [
        {'id': f'location {i+1}', 'depot_id': 'depot 1', 'point': l['point'], 'time_window': l['time_window']} for i, l in enumerate(initial_request['locations'])
    ]
    
    vehicles = [
        {'id': f'courier {i+1}', 'return_to_depot': False} for i, v in enumerate(initial_request['vehicles'])
    ]
    
    depots = [
        {'id': f'depot {i+1}', 'point': d['point'], 'time_window': '00:00:00-2.00:00:00'} for i, d in enumerate(initial_request['depots'])
    ]
    
    options = {
        'time_zone': 3
    }
    
    new_request = {
        'locations': locations,
        'vehicles': vehicles,
        'depots': depots,
        'options': options
    }
    
    return new_request

In [21]:
transformed_request = transform_request(initial_request)

In [22]:
my_utils.save_json(transformed_request, request_path)